In [1]:
from collections import deque, defaultdict
from itertools import combinations, product
from toolz import concat, take
import numpy as np
import networkx as nx
import math


In [2]:
NESW = [0+1j, 1+0j, 0-1j, -1+0j]

def parse_data(txt):
    recs = txt.split('\n\n')
    return [parse_rec(rec) for rec in recs]

def parse_rec(rec):
    lines = [y for y in (x.strip() for x in rec.split('\n')) if y]
    label = lines[0].split()[1][:-1]
    edges = [
        # N, E, S, W
        lines[1],
        ''.join(x[-1] for x in lines[1:]),
        lines[-1],
        ''.join(x[0] for x in lines[1:])
    ]
    return (label, edges)

In [3]:
def encode_edges(edges):
    return [int(e.replace('#', '1').replace('.', '0'), 2) for e in edges]

In [4]:
label, edges = parse_rec(
"""\
Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###
""")
label, edges

('2311', ['..##.#..#.', '...#.##..#', '..###..###', '.#####..#.'])

In [5]:
encode_edges(edges)

[210, 89, 231, 498]

In [6]:
testtxt = """\
Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###...
"""
testdata = parse_data(testtxt)

In [7]:
def reflect_v(t):
    t[0], t[2] = t[2], t[0]
    t[1] = ''.join(reversed(t[1]))
    t[3] = ''.join(reversed(t[3]))

def reflect_h(t):
    t[1], t[3] = t[3], t[1]
    t[0] = ''.join(reversed(t[0]))
    t[2] = ''.join(reversed(t[2]))

def list_orientations(tile):
    L = []
    dq = deque(tile)
    for _ in range(4):
        L.append(tuple(dq)) # N, 1j
        reflect_v(dq)
        L.append(tuple(dq)) # E, 1
        reflect_h(dq)
        L.append(tuple(dq)) # S, -1j
        reflect_v(dq)
        L.append(tuple(dq)) # W, -1
        dq.rotate()
    return L

def load_data(data):
    tiles = np.empty((len(data), 16, 4), dtype=int)
    labels = []
    for i, rec in enumerate(data):
        label, tile = rec
        labels.append(label)
        for j, orient in enumerate(list_orientations(tile)):
            for k, side in enumerate(orient):
                tiles[i, j, k] = int(side.replace('#', '1').replace('.', '0'), 2)
    return tiles, labels

In [8]:
tiles, labels = load_data(testdata)

In [9]:
tiles

array([[[210,  89, 231, 498],
        [231, 616, 210, 318],
        [924, 318, 300, 616],
        [300, 498, 924,  89],
        [ 89, 300, 498, 924],
        [498, 210,  89, 231],
        [318, 231, 616, 210],
        [616, 924, 318, 300],
        [300, 616, 924, 318],
        [924,  89, 300, 498],
        [231, 498, 210,  89],
        [210, 318, 231, 616],
        [616, 210, 318, 231],
        [318, 300, 616, 924],
        [498, 924,  89, 300],
        [ 89, 231, 498, 210]],

       [[710, 498, 564, 841],
        [564, 318, 710, 587],
        [177, 587, 397, 318],
        [397, 841, 177, 498],
        [498, 397, 841, 177],
        [841, 710, 498, 564],
        [587, 564, 318, 710],
        [318, 177, 587, 397],
        [397, 318, 177, 587],
        [177, 498, 397, 841],
        [564, 841, 710, 498],
        [710, 587, 564, 318],
        [318, 710, 587, 564],
        [587, 397, 318, 177],
        [841, 177, 498, 397],
        [498, 564, 841, 710]],

       [[966, 288,  24, 902],
      

In [10]:
labels

['2311', '1951', '1171', '1427', '1489', '2473', '2971', '2729', '3079']

In [11]:
grid = {}

In [12]:
ijks = list((i,j,k) for (i,j,k) in zip(*np.where(tiles==210)) if i != 0 and k == 2)
ijks

[(3, 0, 2), (3, 11, 2)]

In [13]:
tiles[(3,0,2)], tiles[3,0,2], tiles[3][0][2]

(210, 210, 210)

In [14]:
def add_tile(pos, i, j, grid, tiles):
    sides = {c: v for (c, v) in zip(NESW, tiles[i, j])}
    grid[pos] = (i, j, sides)

In [15]:
grid = {}

In [16]:
add_tile(0, 0, 0, grid, tiles)

In [17]:
grid

{0: (0, 0, {1j: 210, (1+0j): 89, -1j: 231, (-1+0j): 498})}

In [18]:
pow(9, 0.5)

3.0

In [19]:
def open_positions(grid, n):
    if not grid:
        return []
    
    sidelen = math.sqrt(n)
    xs = list(int(k.real) for k in grid)
    ys = list(int(k.imag) for k in grid)
    xmin, xmax = min(xs), max(xs)
    ymin, ymax = min(ys), max(ys)
   
    openpos = defaultdict(set)
    for k in grid:
        for drxn in NESW:
            newk = k + drxn
            if newk in grid:
                continue

            # stay in bounds
            x, y = newk.real, newk.imag
            if max(x, xmax) - min(x, xmin) + 1 > sidelen:
                continue
            if max(y, ymax) - min(y, ymin) + 1 > sidelen:
                continue
            
            for drxn2 in NESW:
                nabe = newk + drxn2
                if nabe in grid:
                    openpos[newk].add(nabe)
                    
    ## Fill inside corners first
    return [e for (e, v) in sorted(openpos.items(), key=lambda x: -len(x[1]))]

In [20]:
def possible_tiles(pos, grid, tiles):
    already_played = set(i for (i, j, s) in grid.values())
    if not already_played:
        return None

    filters = []
    for drxn in NESW:
        nabe_pos = pos + drxn
        nabe = grid.get(nabe_pos)
        if nabe is None:
            continue
        ni, nj, nsides = nabe
        filters.append((NESW.index(drxn), nsides[-drxn]))
    if not filters:
        return None

    tilesets = (set((i, j) for (i, j, k) in zip(*np.where(tiles == val))
                       if k == slot and i not in already_played)
                for slot, val in filters)
    try:
        tileset_intersect = next(tilesets)
    except StopIteration:
        return None
    for ts in tilesets:
        tileset_intersect.intersection_update(ts)
    
    return tileset_intersect

In [21]:
grid = {}
add_tile(0,0,0, grid, tiles)

In [22]:
open_positions(grid, 4)

[1j, (1+0j), -1j, (-1+0j)]

In [23]:
possible_tiles(-1j, grid, tiles)

set()

In [24]:
add_tile(-1-1j, 7, 9, grid, tiles)

In [25]:
grid

{0: (0, 0, {1j: 210, (1+0j): 89, -1j: 231, (-1+0j): 498}),
 (-1-1j): (7, 9, {1j: 397, (1+0j): 576, -1j: 680, (-1+0j): 271})}

In [26]:
def solved(grid, n):
    return len(grid) == n

In [27]:
solved(grid, 4)

False

In [28]:
def save_grid(grid):
    return frozenset((complex(k), a, b) for (k, (a, b, _)) in grid.items())

In [29]:
state = save_grid(grid)
state, len(state)

(frozenset({((-1-1j), 7, 9), (0j, 0, 0)}), 2)

In [30]:
def load_grid(state, tiles):
    grid = {}
    for pos, i, j in state:
        add_tile(pos, i, j, grid, tiles)
    return grid
        

In [31]:
load_grid(state, tiles)

{0j: (0, 0, {1j: 210, (1+0j): 89, -1j: 231, (-1+0j): 498}),
 (-1-1j): (7, 9, {1j: 397, (1+0j): 576, -1j: 680, (-1+0j): 271})}

In [32]:
load_grid([(0,0,0)], tiles)

{0: (0, 0, {1j: 210, (1+0j): 89, -1j: 231, (-1+0j): 498})}

In [33]:
def dfs(tiles, n, initial_state=[(0,0,0)]):
    grid = load_grid(initial_state, tiles)
    initial = save_grid(grid)
    frontier = [initial]
    explored = {initial}
    
    counter = 0
    while frontier:
        state = frontier.pop()
        if len(state) == n:
            print("counter =", counter)
            return state
        grid = load_grid(state, tiles)
        for pos in open_positions(grid, n):
            for (i, j) in possible_tiles(pos, grid, tiles):
                add_tile(pos, i, j, grid, tiles)
                newstate = save_grid(grid)
                del grid[pos]
                if newstate in explored:
                    continue
                explored.add(newstate)
                frontier.append(newstate)
        counter += 1
    print("counter =", counter)

In [34]:
dfs(tiles, 4)

counter = 9


frozenset({((-1+0j), 1, 0), ((-1+1j), 7, 0), (0j, 0, 0), (1j, 3, 0)})

In [35]:
%%time
completed_puzzle = dfs(tiles, 9)
completed_puzzle

counter = 1223
CPU times: user 227 ms, sys: 0 ns, total: 227 ms
Wall time: 227 ms


In [36]:
len(labels)

9

In [37]:
%%time
for i in range(16):
    puzzle = dfs(tiles, 9, [(0, 0, i)])
    if puzzle:
        print(i)
        print(puzzle)
        break

counter = 1223
counter = 1223
counter = 1223
counter = 1223
counter = 1223
counter = 1223
counter = 1223
counter = 1223
counter = 1223
counter = 1223
counter = 1223
counter = 1223
counter = 1223
counter = 1223
counter = 1223
counter = 1223
CPU times: user 3.14 s, sys: 5.26 ms, total: 3.15 s
Wall time: 3.14 s


In [38]:
def bfs(tiles, n, initial_state=[(0,0,0)]):
    grid = load_grid(initial_state, tiles)
    initial = save_grid(grid)
    frontier = deque([initial])
    explored = {initial}
    
    ctr = 0
    while frontier:
        state = frontier.popleft()
        if len(state) == n:
            print("count =", ctr)
            return state
        grid = load_grid(state, tiles)
        for pos in open_positions(grid, n):
            for (i, j) in possible_tiles(pos, grid, tiles):
                add_tile(pos, i, j, grid, tiles)
                newstate = save_grid(grid)
                del grid[pos]
                if newstate in explored:
                    continue
                explored.add(newstate)
                frontier.append(newstate)
        ctr += 1
    print("count =", ctr)

In [39]:
%%time
completed_puzzle = bfs(tiles, 4)
completed_puzzle

count = 31
CPU times: user 3.59 ms, sys: 0 ns, total: 3.59 ms
Wall time: 3.29 ms


frozenset({((-1+0j), 1, 0), ((-1+1j), 7, 0), (0j, 0, 0), (1j, 3, 0)})

In [40]:
%%time
for i in range(16):
    puzzle = bfs(tiles, 9, [(0, 0, i)])
    if puzzle:
        print(i)
        print(puzzle)
        break

count = 1223
count = 1223
count = 1223
count = 1223
count = 1223
count = 1223
count = 1223
count = 1223
count = 1223
count = 1223
count = 1223
count = 1223
count = 1223
count = 1223
count = 1223
count = 1223
CPU times: user 3.11 s, sys: 0 ns, total: 3.11 s
Wall time: 3.1 s


In [41]:
labels.index('1951')

1

In [42]:
tile1951_rec = """\
Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#.."""

In [43]:
label, edges = parse_rec(tile1951_rec)
label, edges

('1951', ['#.##...##.', '.#####..#.', '#...##.#..', '##.#..#..#'])

In [44]:
orientations_1951 = list_orientations(edges)
orientations_1951

[('#.##...##.', '.#####..#.', '#...##.#..', '##.#..#..#'),
 ('#...##.#..', '.#..#####.', '#.##...##.', '#..#..#.##'),
 ('..#.##...#', '#..#..#.##', '.##...##.#', '.#..#####.'),
 ('.##...##.#', '##.#..#..#', '..#.##...#', '.#####..#.'),
 ('.#####..#.', '.##...##.#', '##.#..#..#', '..#.##...#'),
 ('##.#..#..#', '#.##...##.', '.#####..#.', '#...##.#..'),
 ('#..#..#.##', '#...##.#..', '.#..#####.', '#.##...##.'),
 ('.#..#####.', '..#.##...#', '#..#..#.##', '.##...##.#'),
 ('.##...##.#', '.#..#####.', '..#.##...#', '#..#..#.##'),
 ('..#.##...#', '.#####..#.', '.##...##.#', '##.#..#..#'),
 ('#...##.#..', '##.#..#..#', '#.##...##.', '.#####..#.'),
 ('#.##...##.', '#..#..#.##', '#...##.#..', '.#..#####.'),
 ('.#..#####.', '#.##...##.', '#..#..#.##', '#...##.#..'),
 ('#..#..#.##', '.##...##.#', '.#..#####.', '..#.##...#'),
 ('##.#..#..#', '..#.##...#', '.#####..#.', '.##...##.#'),
 ('.#####..#.', '#...##.#..', '##.#..#..#', '#.##...##.')]

In [45]:
[encode_edges(x) for x in orientations_1951]

[[710, 498, 564, 841],
 [564, 318, 710, 587],
 [177, 587, 397, 318],
 [397, 841, 177, 498],
 [498, 397, 841, 177],
 [841, 710, 498, 564],
 [587, 564, 318, 710],
 [318, 177, 587, 397],
 [397, 318, 177, 587],
 [177, 498, 397, 841],
 [564, 841, 710, 498],
 [710, 587, 564, 318],
 [318, 710, 587, 564],
 [587, 397, 318, 177],
 [841, 177, 498, 397],
 [498, 564, 841, 710]]

In [46]:
encode_edges(edges)

[710, 498, 564, 841]

In [47]:
%%time
bfs(tiles, 9, [(0+0j,1,1)])

count = 1163
CPU times: user 182 ms, sys: 0 ns, total: 182 ms
Wall time: 182 ms


In [48]:
%%time
dfs(tiles, 9, [(0,1,1)])

counter = 1163
CPU times: user 178 ms, sys: 0 ns, total: 178 ms
Wall time: 177 ms


In [49]:
grid = {}

In [50]:
add_tile(0, 1, 1, grid, tiles)
grid

{0: (1, 1, {1j: 564, (1+0j): 318, -1j: 710, (-1+0j): 587})}

In [51]:
possible_tiles(1, grid, tiles)

{(0, 1), (0, 8)}

In [52]:
labels[0]

'2311'

In [53]:
testdata[0]

('2311', ['..##.#..#.', '...#.##..#', '..###..###', '.#####..#.'])

In [54]:
tstrecs = testtxt.split('\n\n')

In [55]:
tstrecs[0]

'Tile 2311:\n..##.#..#.\n##..#.....\n#...##..#.\n####.#...#\n##.##.###.\n##...#.###\n.#.#.#..##\n..#....#..\n###...#.#.\n..###..###'

In [56]:
label, edges = parse_rec(tstrecs[0])

In [57]:
edges

['..##.#..#.', '...#.##..#', '..###..###', '.#####..#.']

In [58]:
list(enumerate(list_orientations(edges)))

[(0, ('..##.#..#.', '...#.##..#', '..###..###', '.#####..#.')),
 (1, ('..###..###', '#..##.#...', '..##.#..#.', '.#..#####.')),
 (2, ('###..###..', '.#..#####.', '.#..#.##..', '#..##.#...')),
 (3, ('.#..#.##..', '.#####..#.', '###..###..', '...#.##..#')),
 (4, ('...#.##..#', '.#..#.##..', '.#####..#.', '###..###..')),
 (5, ('.#####..#.', '..##.#..#.', '...#.##..#', '..###..###')),
 (6, ('.#..#####.', '..###..###', '#..##.#...', '..##.#..#.')),
 (7, ('#..##.#...', '###..###..', '.#..#####.', '.#..#.##..')),
 (8, ('.#..#.##..', '#..##.#...', '###..###..', '.#..#####.')),
 (9, ('###..###..', '...#.##..#', '.#..#.##..', '.#####..#.')),
 (10, ('..###..###', '.#####..#.', '..##.#..#.', '...#.##..#')),
 (11, ('..##.#..#.', '.#..#####.', '..###..###', '#..##.#...')),
 (12, ('#..##.#...', '..##.#..#.', '.#..#####.', '..###..###')),
 (13, ('.#..#####.', '.#..#.##..', '#..##.#...', '###..###..')),
 (14, ('.#####..#.', '###..###..', '...#.##..#', '.#..#.##..')),
 (15, ('...#.##..#', '..###..###', 

In [59]:
add_tile(1, 0, 1, grid, tiles)
grid

{0: (1, 1, {1j: 564, (1+0j): 318, -1j: 710, (-1+0j): 587}),
 1: (0, 1, {1j: 231, (1+0j): 616, -1j: 210, (-1+0j): 318})}

In [60]:
open_positions(grid, 9)

[1j, -1j, (-1+0j), (1+1j), (2+0j), (1-1j)]

In [61]:
possible_tiles(1-1j, grid, tiles)

{(3, 1), (3, 10)}

In [62]:
print(tstrecs[3])

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.


In [63]:
label, edges = parse_rec(tstrecs[3])

In [64]:
edges

['###.##.#..', '..###.#.#.', '..##.#..#.', '#..#......']

In [65]:
list(enumerate(list_orientations(edges)))

[(0, ('###.##.#..', '..###.#.#.', '..##.#..#.', '#..#......')),
 (1, ('..##.#..#.', '.#.#.###..', '###.##.#..', '......#..#')),
 (2, ('.#..#.##..', '......#..#', '..#.##.###', '.#.#.###..')),
 (3, ('..#.##.###', '#..#......', '.#..#.##..', '..###.#.#.')),
 (4, ('..###.#.#.', '..#.##.###', '#..#......', '.#..#.##..')),
 (5, ('#..#......', '###.##.#..', '..###.#.#.', '..##.#..#.')),
 (6, ('......#..#', '..##.#..#.', '.#.#.###..', '###.##.#..')),
 (7, ('.#.#.###..', '.#..#.##..', '......#..#', '..#.##.###')),
 (8, ('..#.##.###', '.#.#.###..', '.#..#.##..', '......#..#')),
 (9, ('.#..#.##..', '..###.#.#.', '..#.##.###', '#..#......')),
 (10, ('..##.#..#.', '#..#......', '###.##.#..', '..###.#.#.')),
 (11, ('###.##.#..', '......#..#', '..##.#..#.', '.#.#.###..')),
 (12, ('.#.#.###..', '###.##.#..', '......#..#', '..##.#..#.')),
 (13, ('......#..#', '..#.##.###', '.#.#.###..', '.#..#.##..')),
 (14, ('#..#......', '.#..#.##..', '..###.#.#.', '..#.##.###')),
 (15, ('..###.#.#.', '..##.#..#.', 

In [66]:
add_tile(1-1j, 3, 1, grid, tiles)
grid

{0: (1, 1, {1j: 564, (1+0j): 318, -1j: 710, (-1+0j): 587}),
 1: (0, 1, {1j: 231, (1+0j): 616, -1j: 210, (-1+0j): 318}),
 (1-1j): (3, 1, {1j: 210, (1+0j): 348, -1j: 948, (-1+0j): 9})}

In [67]:
open_positions(grid, 9)

[-1j, 1j, (-1+0j), (1+1j), (2+0j), (2-1j), (1-2j)]

In [68]:
possible_tiles(-1j, grid, tiles)

{(7, 1)}

In [69]:
print(tstrecs[7])

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.


In [70]:
label, edges = parse_rec(tstrecs[7])

In [71]:
edges

['...#.#.#.#', '#..#......', '#.##...##.', '.#....####']

In [72]:
list(enumerate(list_orientations(edges)))

[(0, ('...#.#.#.#', '#..#......', '#.##...##.', '.#....####')),
 (1, ('#.##...##.', '......#..#', '...#.#.#.#', '####....#.')),
 (2, ('.##...##.#', '####....#.', '#.#.#.#...', '......#..#')),
 (3, ('#.#.#.#...', '.#....####', '.##...##.#', '#..#......')),
 (4, ('#..#......', '#.#.#.#...', '.#....####', '.##...##.#')),
 (5, ('.#....####', '...#.#.#.#', '#..#......', '#.##...##.')),
 (6, ('####....#.', '#.##...##.', '......#..#', '...#.#.#.#')),
 (7, ('......#..#', '.##...##.#', '####....#.', '#.#.#.#...')),
 (8, ('#.#.#.#...', '......#..#', '.##...##.#', '####....#.')),
 (9, ('.##...##.#', '#..#......', '#.#.#.#...', '.#....####')),
 (10, ('#.##...##.', '.#....####', '...#.#.#.#', '#..#......')),
 (11, ('...#.#.#.#', '####....#.', '#.##...##.', '......#..#')),
 (12, ('......#..#', '...#.#.#.#', '####....#.', '#.##...##.')),
 (13, ('####....#.', '#.#.#.#...', '......#..#', '.##...##.#')),
 (14, ('.#....####', '.##...##.#', '#..#......', '#.#.#.#...')),
 (15, ('#..#......', '#.##...##.', 

In [73]:
add_tile(0-1j, 7, 1, grid, tiles)
grid

{0: (1, 1, {1j: 564, (1+0j): 318, -1j: 710, (-1+0j): 587}),
 1: (0, 1, {1j: 231, (1+0j): 616, -1j: 210, (-1+0j): 318}),
 (1-1j): (3, 1, {1j: 210, (1+0j): 348, -1j: 948, (-1+0j): 9}),
 -1j: (7, 1, {1j: 710, (1+0j): 9, -1j: 85, (-1+0j): 962})}

In [89]:
def bfs(tiles, n, initial_state=[(0,0,0)]):
    grid = load_grid(initial_state, tiles)
    initial = save_grid(grid)
    frontier = deque([initial])
    explored = {initial}
    log = []
    
    while frontier:
        state = frontier.popleft()
        log.append(state)
        if len(state) == n:
            break
        grid = load_grid(state, tiles)
        for pos in open_positions(grid, n):
            for (i, j) in possible_tiles(pos, grid, tiles):
                add_tile(pos, i, j, grid, tiles)
                newstate = save_grid(grid)
                del grid[pos]
                if newstate in explored:
                    continue
                explored.add(newstate)
                frontier.append(newstate)
    else:
        print('failed')
    return (state, log)


In [90]:
%%time
state, log = bfs(tiles, 9, [(0, 1, 1)])

failed
CPU times: user 172 ms, sys: 0 ns, total: 172 ms
Wall time: 171 ms


In [91]:
len(state)

8

In [92]:
state

frozenset({((1+0j), 0, 8),
           ((1-2j), 4, 8),
           ((2+0j), 8, 0),
           ((2-1j), 5, 7),
           ((2-2j), 2, 3),
           (-1j, 7, 1),
           (-2j, 6, 1),
           (0j, 1, 1)})

In [96]:
state.pop()

AttributeError: 'frozenset' object has no attribute 'pop'

In [97]:
set(state).pop()

((2-1j), 5, 7)

In [98]:
sorted(state, key=lambda x: (-x[0].imag, x[0].real))

[(0j, 1, 1),
 ((1+0j), 0, 8),
 ((2+0j), 8, 0),
 (-1j, 7, 1),
 ((2-1j), 5, 7),
 (-2j, 6, 1),
 ((1-2j), 4, 8),
 ((2-2j), 2, 3)]

In [95]:
len(log)

1163

In [100]:
[sorted(s, key=lambda x: (-x[0].imag, x[0].real)) for s in log[-10:]]

[[(0j, 1, 1),
  ((1+0j), 0, 1),
  ((2+0j), 8, 0),
  (-1j, 7, 10),
  ((2-1j), 5, 7),
  (-2j, 6, 1),
  ((1-2j), 4, 8),
  ((2-2j), 2, 3)],
 [(0j, 1, 1),
  ((1+0j), 0, 1),
  ((2+0j), 8, 0),
  (-1j, 7, 1),
  ((2-1j), 5, 7),
  (-2j, 6, 1),
  ((1-2j), 4, 8),
  ((2-2j), 2, 3)],
 [(0j, 1, 1),
  ((1+0j), 0, 8),
  ((2+0j), 8, 0),
  (-1j, 7, 10),
  ((2-1j), 5, 12),
  (-2j, 6, 1),
  ((1-2j), 4, 1),
  ((2-2j), 2, 3)],
 [(0j, 1, 1),
  ((1+0j), 0, 8),
  ((2+0j), 8, 0),
  (-1j, 7, 1),
  ((2-1j), 5, 12),
  (-2j, 6, 1),
  ((1-2j), 4, 1),
  ((2-2j), 2, 3)],
 [(0j, 1, 1),
  ((1+0j), 0, 8),
  ((2+0j), 8, 0),
  (-1j, 7, 10),
  ((2-1j), 5, 12),
  (-2j, 6, 1),
  ((1-2j), 4, 8),
  ((2-2j), 2, 3)],
 [(0j, 1, 1),
  ((1+0j), 0, 8),
  ((2+0j), 8, 0),
  (-1j, 7, 1),
  ((2-1j), 5, 12),
  (-2j, 6, 1),
  ((1-2j), 4, 8),
  ((2-2j), 2, 3)],
 [(0j, 1, 1),
  ((1+0j), 0, 8),
  ((2+0j), 8, 0),
  (-1j, 7, 10),
  ((2-1j), 5, 7),
  (-2j, 6, 1),
  ((1-2j), 4, 1),
  ((2-2j), 2, 3)],
 [(0j, 1, 1),
  ((1+0j), 0, 8),
  ((2+0j), 8, 0

In [102]:
grid = load_grid(log[-1], tiles)
grid

{(2-1j): (5, 7, {1j: 184, (1+0j): 348, -1j: 399, (-1+0j): 481}),
 -2j: (6, 1, {1j: 85, (1+0j): 689, -1j: 161, (-1+0j): 78}),
 -1j: (7, 1, {1j: 710, (1+0j): 9, -1j: 85, (-1+0j): 962}),
 (1-2j): (4, 8, {1j: 43, (1+0j): 288, -1j: 183, (-1+0j): 689}),
 (2-2j): (2, 3, {1j: 399, (1+0j): 902, -1j: 96, (-1+0j): 288}),
 (1+0j): (0, 8, {1j: 300, (1+0j): 616, -1j: 924, (-1+0j): 318}),
 (2+0j): (8, 0, {1j: 702, (1+0j): 264, -1j: 184, (-1+0j): 616}),
 0j: (1, 1, {1j: 564, (1+0j): 318, -1j: 710, (-1+0j): 587})}

In [106]:
def gsort(x):
    return (-x[0].imag, x[0].real)

In [107]:
sorted(grid.items(), key=gsort)

[(0j, (1, 1, {1j: 564, (1+0j): 318, -1j: 710, (-1+0j): 587})),
 ((1+0j), (0, 8, {1j: 300, (1+0j): 616, -1j: 924, (-1+0j): 318})),
 ((2+0j), (8, 0, {1j: 702, (1+0j): 264, -1j: 184, (-1+0j): 616})),
 (-1j, (7, 1, {1j: 710, (1+0j): 9, -1j: 85, (-1+0j): 962})),
 ((2-1j), (5, 7, {1j: 184, (1+0j): 348, -1j: 399, (-1+0j): 481})),
 (-2j, (6, 1, {1j: 85, (1+0j): 689, -1j: 161, (-1+0j): 78})),
 ((1-2j), (4, 8, {1j: 43, (1+0j): 288, -1j: 183, (-1+0j): 689})),
 ((2-2j), (2, 3, {1j: 399, (1+0j): 902, -1j: 96, (-1+0j): 288}))]

In [109]:
open_positions(grid, 9)

[(1-1j)]

In [110]:
possible_tiles(1-1j, grid, tiles)

set()

In [112]:
from collections import Counter

In [113]:
Counter(len(x) for x in log).most_common()

[(6, 409), (7, 338), (5, 282), (4, 89), (3, 20), (8, 20), (2, 4), (1, 1)]

In [117]:
gridlabels ={x: labels[y[0]] for (x, y) in grid.items()}
sorted(gridlabels.items(), key=gsort)

[(0j, '1951'),
 ((1+0j), '2311'),
 ((2+0j), '3079'),
 (-1j, '2729'),
 ((2-1j), '2473'),
 (-2j, '2971'),
 ((1-2j), '1489'),
 ((2-2j), '1171')]

In [118]:
def get_gridlabels(grid):
    return {x: labels[y[0]] for (x, y) in grid.items()}

In [119]:
get_gridlabels(grid)

{(2-1j): '2473',
 -2j: '2971',
 -1j: '2729',
 (1-2j): '1489',
 (2-2j): '1171',
 (1+0j): '2311',
 (2+0j): '3079',
 0j: '1951'}

In [120]:
log[-1]

frozenset({((1+0j), 0, 8),
           ((1-2j), 4, 8),
           ((2+0j), 8, 0),
           ((2-1j), 5, 7),
           ((2-2j), 2, 3),
           (-1j, 7, 1),
           (-2j, 6, 1),
           (0j, 1, 1)})

In [121]:
def get_loglabels(logentry):
    return {x:y for (x,y,z) in logentry}

In [123]:
get_loglabels(log[-1]) == get_loglabels(log[-2])

True

In [124]:
lastloglabels = get_loglabels(log[-1])
lastloglabels

{(2-1j): 5, -2j: 6, -1j: 7, (1-2j): 4, (2-2j): 2, (1+0j): 0, (2+0j): 8, 0j: 1}

In [125]:
sum(get_loglabels(x) == lastloglabels for x in log)

16

In [126]:
[x for x in log if get_loglabels(x) == lastloglabels]

[frozenset({((1+0j), 0, 1),
            ((1-2j), 4, 1),
            ((2+0j), 8, 0),
            ((2-1j), 5, 12),
            ((2-2j), 2, 3),
            (-1j, 7, 10),
            (-2j, 6, 1),
            (0j, 1, 1)}),
 frozenset({((1+0j), 0, 1),
            ((1-2j), 4, 1),
            ((2+0j), 8, 0),
            ((2-1j), 5, 12),
            ((2-2j), 2, 3),
            (-1j, 7, 1),
            (-2j, 6, 1),
            (0j, 1, 1)}),
 frozenset({((1+0j), 0, 1),
            ((1-2j), 4, 8),
            ((2+0j), 8, 0),
            ((2-1j), 5, 12),
            ((2-2j), 2, 3),
            (-1j, 7, 10),
            (-2j, 6, 1),
            (0j, 1, 1)}),
 frozenset({((1+0j), 0, 1),
            ((1-2j), 4, 8),
            ((2+0j), 8, 0),
            ((2-1j), 5, 12),
            ((2-2j), 2, 3),
            (-1j, 7, 1),
            (-2j, 6, 1),
            (0j, 1, 1)}),
 frozenset({((1+0j), 0, 1),
            ((1-2j), 4, 1),
            ((2+0j), 8, 0),
            ((2-1j), 5, 7),
            ((2-2j

In [134]:
def lookup_tile(txt, tiles=tiles, labels=labels):
    label, edges = parse_rec(txt)
    i = labels.index(label)
    
    return i, edges

In [132]:
tiletxt = """\
 
Tile 1951:
#...##.#..
..#.#..#.#
.###....#.
###.##.##.
.###.#####
.##.#....#
#...######
.....#..##
#.####...#
#.##...##.


"""

In [135]:
lookup_tile(tiletxt)

(1, ['#...##.#..', '.#..#####.', '#.##...##.', '#..#..#.##'])